<a href="https://colab.research.google.com/github/ManavSarkar/Weight-Prediction-using-Machine-Learning/blob/deep-feature-extraction/Deep_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
! rm -rf datasets

In [51]:
! git init

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [69]:
! git remote set-url origin https://github.com/ManavSarkar/Weight-Prediction-using-Machine-Learning.git

In [71]:
! git config --global user.name "Manav Sarkar"
! git config --global user.email "manavsarkar07@gmail.com"

In [70]:
! git switch -c deep-feature-extraction

fatal: A branch named 'deep-feature-extraction' already exists.


In [72]:
! git pull https://github.com/ManavSarkar/Weight-Prediction-using-Machine-Learning.git

fatal: could not read Username for 'https://github.com': No such device or address


In [4]:
dataset_path = 'datasets/data.csv'
import pandas as pd

df = pd.read_csv(dataset_path)


In [5]:
df.head()

,img_name,sex,age,height,weight
0,000002_F_18_165100_8482178.jpg,0,18,1.6510,84.82178
1,000003_F_18_170180_3991613.jpg,0,18,1.7018,39.91613
2,000003_F_18_170180_7257478.jpg,0,18,1.7018,72.57478
3,000004_F_22_157480_5606402.jpg,0,22,1.5748,56.06402
4,000004_F_22_157480_6245967.jpg,0,22,1.5748,62.45967


In [6]:
# first getting ResNet 101 model from torch
import torch
resnet_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)

Using cache found in /home/codespace/.cache/torch/hub/pytorch_vision_v0.10.0


In [7]:
# replacing last layer with 15 features instead of 1000
import torch
import torch.nn as nn
resnet_model.fc = nn.Linear(2048, 15)

In [8]:
# using the adam optimizer
import torch.optim as optim
optimizer = optim.Adam(resnet_model.parameters(), lr=0.001)

In [6]:
print(resnet_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
# scale the images to 224x224
from torchvision import transforms
from PIL import Image
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

# get the image from the path and apply the transform
def get_image(path):
    img = Image.open(path)
    tensor = transform(img)
    tensor = torch.unsqueeze(tensor, 0)
    return tensor

# all images path
IMAGES_PATH = 'datasets/Images/'

# get the image tensor
def get_image_tensor(image_name):
    image_path = IMAGES_PATH + image_name
    image_tensor = get_image(image_path)
    return image_tensor

# images name in df first column
images_name = df.iloc[:, 0]

# get the image tensor for all images
images_tensor = []

for image_name in images_name:
    temp = get_image_tensor(image_name)
    images_tensor.append(temp)

# get the features from the images
def get_features(image_tensor):
    features = resnet_model(image_tensor)
    return features
print(images_tensor[0])


tensor([[[[ 0.6049,  0.6221,  0.9132,  ..., -1.0733,  0.1939,  0.7077],
          [ 0.6049,  0.5878,  0.9132,  ..., -0.8164, -0.2684,  0.7419],
          [ 0.6049,  0.5878,  0.9132,  ..., -0.4739, -0.6794,  0.7419],
          ...,
          [-1.1589, -1.1589, -0.6281,  ...,  1.4269,  1.4098,  0.6563],
          [-1.0904, -1.1589, -0.6109,  ...,  1.7180,  1.7865,  1.2557],
          [-1.0562, -1.0733, -0.5596,  ...,  1.5468,  1.3070,  1.2385]],

         [[ 0.4853,  0.5203,  0.9405,  ..., -1.2129,  0.1001,  0.6078],
          [ 0.4853,  0.4853,  0.9405,  ..., -0.9503, -0.3725,  0.6254],
          [ 0.4853,  0.4853,  0.9230,  ..., -0.5826, -0.8102,  0.6429],
          ...,
          [-0.9328, -0.9678, -0.4776,  ...,  1.2206,  1.1681,  0.3978],
          [-0.8803, -0.8978, -0.4251,  ...,  1.5707,  1.6408,  1.0805],
          [-0.6527, -0.6527, -0.2325,  ...,  1.4132,  1.1856,  1.0980]],

         [[ 0.2173,  0.2173,  0.8448,  ..., -1.1421, -0.0615,  0.4788],
          [ 0.1999,  0.1999,  

In [10]:
print(get_features(images_tensor[0]))

tensor([[ 0.3234,  0.1626,  0.2285,  0.1816,  0.2188, -0.0875, -0.7718, -0.0454,
          0.2098, -0.1108, -0.2122,  0.0680,  0.1643,  0.0478, -0.0153]],
       grad_fn=<AddmmBackward0>)
